In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from pasna_analysis import DataLoader, Experiment, utils

experiment_name = '20240619'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(DataLoader(exp_path), first_peak_threshold=0)

In [ ]:
'''Plots activity signals for a group of embryos.'''
# Change start and end to select different groups
start = 0
end = 3

fig, ax = plt.subplots(end-start, 2, figsize=(12, 4*(end-start)))
for i, emb in enumerate(exp.embryos[start:end]):
    time = emb.activity[:, 0]
    trace = exp.traces[emb.name]
    ax[i][0].set_title(f'{emb.name} - GCamP')
    ax[i][0].plot(time, trace.active, color='green')
    ax[i][1].set_title(f'{emb.name} - tdTomato')
    ax[i][1].plot(time, trace.struct, color='firebrick')

    for peak in trace.peak_times:
        ax[i][0].axvline(peak, color='k')

In [ ]:
'''Plots dff for a group of embryos.'''
# Change start and end to select different groups
start = 0
end = 6

fig, ax = plt.subplots(end-start, figsize=(8, 4*(end-start)))
for i, emb in enumerate(exp.embryos[start:end]):
    time = emb.activity[:, 0]
    trace = exp.traces[emb.name]
    time = time[:trace.trim_idx]
    ax[i].set_title(f'{emb.name} - dff')
    ax[i].plot(time, trace.dff)
    for peak in trace.peak_times:
        ax[i].axvline(peak, color='k', alpha=0.3)

In [ ]:
'''Plots AUC, grouped by bins.'''
data = {'auc': [], 'bin': [], 'emb': []}

n_bins = 5
first_bin = 2
bin_width = 0.2
for i, emb in enumerate(exp.embryos):
    trace = exp.traces[emb.name]

    dev_time_at_peaks = emb.get_DT_from_time(trace.peak_times)

    bins = [first_bin + j*bin_width for j in range(n_bins)]
    bin_idxs = utils.split_in_bins(dev_time_at_peaks, bins)
    data['auc'].extend(trace.peak_aucs)
    data['bin'].extend(bin_idxs)
    data['emb'].extend([str(i)]*len(trace.peak_aucs))

# add a last bin point to generate the labels
bins.append(first_bin + bin_width*n_bins)
x_labels = [f"{s}~{e}" for (s, e) in zip(bins[:-1], bins[1:])]
ax = sns.pointplot(data=data, x='bin', y='auc', linestyle='None')
ax.set_xticks(ticks=list(range(n_bins)), labels=x_labels)
ax.set_title(f'Binned AUC - Exp {exp.name}')
ax.set_ylabel('AUC [activity*t]');

In [ ]:
'''Plot peak times for all embryos. This should help to isolate the 
early peaks.'''
times = []
for emb in exp.embryos:
    trace = exp.traces[emb.name]
    times.append([t for t in trace.peak_times if t < 15000])

fig, ax = plt.subplots()
for i, time in enumerate(times):
    ax.plot(time, [i]*len(time), marker='.', linestyle='dashed', linewidth=0.5)

ax.vlines(1800, 0, len(exp.embryos), color='k', linewidth=0.3)
ax.set_title(f'Peak times for exp {exp.name}')


In [ ]:
'''Compare first interspike interval and average interspike interval'''
import numpy as np

avg_ISIs = []
first_ISIs = []
for emb in exp.embryos:
    trace = exp.traces[emb.name]
    peak_times = trace.peak_times
    avg_ISIs.append(np.average(np.diff(peak_times[1:])))
    first_ISIs.append(peak_times[1] - peak_times[0])

max_val = np.max((np.max(avg_ISIs), np.max(first_ISIs)))
diag = list(range(0, int(max_val), 50))

fig, ax = plt.subplots()
ax.plot(avg_ISIs, first_ISIs, 'k.')
ax.plot(diag, diag, label='y=x')
ax.plot(diag, list(range(0, int(max_val)*2, 50*2)), label='y=2*x')
ax.set_xlabel('average ISI')
ax.set_ylabel('first ISI')
ax.set_xlim(0, max_val)
ax.set_ylim(0, max_val)
ax.set_title(f'Differences between first and avg intervals - exp {exp.name}')
ax.legend();